In [1]:
from utils import load_data
import pandas as pd
from autogluon.tabular import TabularPredictor
import pandas as pd
import json
import h5py

/home/julian/miniconda3/envs/dtiam-esm/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

def load_embeddings(file_path, unique_keys=None):
    with h5py.File(file_path, "r") as f:
        data = f["data"][()]
        keys = [k.decode("utf-8") for k in f["keys"][()]]
        data_dict_loaded = dict(zip(keys, data))
    if unique_keys is not None:
        new_dict = {key: data_dict_loaded[key] for key in unique_keys}
        return new_dict
    return data_dict_loaded

In [3]:
dataset = 'kiba'
# setting = 'warm_start'
setting = 'drug_coldstart'
fold = 0
dataset_path = "../data/dta/" + dataset
folds_path = dataset_path + f"/data_folds/{setting}/"
# fpath = f'/home/julian/DTIAM/data/dta/{dataset}/'

In [4]:
import pickle
# comp_feat = pickle.load(open(dataset_path + "/features/compound_features.pkl", "rb"))
prot_feat = pickle.load(open(dataset_path + "/features/protein_features.pkl", "rb"))

In [5]:
train = pd.read_csv(folds_path + "train_fold_" + str(fold) + ".csv")
test = pd.read_csv(folds_path + "test_fold_" + str(fold) + ".csv")

ligands = json.load(open(dataset_path + "/ligands_can.txt"))
proteins = json.load(open(dataset_path + "/proteins.txt"))

# Drug

In [6]:
train['smiles'] = train['drug_id'].apply(lambda x: ligands[x])
test['smiles'] = test['drug_id'].apply(lambda x: ligands[x])
MolE_embs = load_embeddings('/home/julian/mole_embed/notebooks/DTI_benchmark/prot_mols_embeddings/MolE_GuacaMol_27113.ckpt.h5')
train['drub_emb'] = train['smiles'].apply(lambda x: MolE_embs[x])
test['drub_emb'] = test['smiles'].apply(lambda x: MolE_embs[x])
train.drop(columns=['drug_id', 'smiles'], inplace=True)
test.drop(columns=['drug_id', 'smiles'], inplace=True)

# Protein

In [7]:
train['prot_emb'] = train['protein_id'].apply(lambda x: prot_feat[x])
test['prot_emb'] = test['protein_id'].apply(lambda x: prot_feat[x])
train.drop(columns=['protein_id'], inplace=True)
test.drop(columns=['protein_id'], inplace=True)

In [8]:
import numpy as np
train_df = train.apply(lambda x: pd.Series(np.hstack([x['drub_emb'], x['prot_emb']])), axis=1)
train_df['y'] = train['affinity']

In [9]:
test_df = test.apply(lambda x: pd.Series(np.hstack([x['drub_emb'], x['prot_emb']])), axis=1)
test_df['y'] = test['affinity']

In [10]:
train_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2039,2040,2041,2042,2043,2044,2045,2046,2047,y
0,-1.486857,-2.852608,2.226141,-0.771469,-0.53247,1.374549,-0.905905,2.068331,-2.270883,-0.709579,...,-0.003497,-0.092343,0.056811,-0.017626,-0.057044,0.038800,-0.113871,0.059128,0.047931,11.1
1,-1.486857,-2.852608,2.226141,-0.771469,-0.53247,1.374549,-0.905905,2.068331,-2.270883,-0.709579,...,0.030705,-0.091796,-0.016181,0.029888,-0.057828,0.048171,-0.059174,0.046706,0.080474,11.1
2,-1.486857,-2.852608,2.226141,-0.771469,-0.53247,1.374549,-0.905905,2.068331,-2.270883,-0.709579,...,0.028138,-0.069517,0.017836,0.021373,-0.023830,0.049003,-0.069897,0.045682,0.043708,11.1
3,-1.486857,-2.852608,2.226141,-0.771469,-0.53247,1.374549,-0.905905,2.068331,-2.270883,-0.709579,...,-0.006736,-0.179605,-0.001505,0.028528,-0.114054,0.034848,-0.234336,0.080385,0.143831,11.1
4,-1.486857,-2.852608,2.226141,-0.771469,-0.53247,1.374549,-0.905905,2.068331,-2.270883,-0.709579,...,0.037864,-0.255422,0.016660,0.003467,-0.127575,0.044249,-0.200034,0.106770,0.148296,11.1


In [11]:
included_model_types = []
eval_metric = None
preset=None
ex_model=[
    # 'RandomForestMSE', 'KNeighborsDist', 'KNeighborsUnif', 'CatBoost'
]
# predictor = TabularPredictor(label="y", eval_metric=eval_metric).fit(
#     train_data=train_df, excluded_model_types=ex_model, presets=preset
# )

In [19]:
# ag-20250616_230514 es drug_coldstart
predictor = TabularPredictor.load('/home/julian/DTIAM/code/AutogluonModels/ag-drug_cold_start_0')

In [21]:
# sorted(predictor.model_names())

['CatBoost',
 'ExtraTreesMSE',
 'KNeighborsDist',
 'KNeighborsUnif',
 'LightGBM',
 'LightGBMXT',
 'NeuralNetFastAI',
 'RandomForestMSE',
 'XGBoost']

In [24]:
predictor.fit(train_data=train_df, excluded_model_types=ex_model, presets=preset, resume=True)

AssertionError: Predictor is already fit! To fit additional models, refer to `predictor.fit_extra`, or create a new `Predictor`.

In [13]:
test_data_nolab = test_df.drop(columns=["y"])

In [14]:
pred_scores = predictor.predict(test_data_nolab)

In [17]:
import numpy as np
from utils import load_data, rmse, mse, pearson, spearman, ci, roc_auc, pr_auc
res_all = pd.DataFrame(columns=["RMSE", "MSE", "Pearson", "Spearman", "CI"])
G, P = np.array(test_df["y"]), np.array(pred_scores)
ret = [rmse(G, P), mse(G, P), pearson(G, P), spearman(G, P), ci(G, P)]

In [18]:
i = 0
print(
    f"fold: {i+1}, RMSE: {ret[0]}, MSE: {ret[1]}, Pearson: {ret[2]}, Spearman: {ret[3]}, CI: {ret[4]}"
)
res_all.loc[i] = ret

fold: 1, RMSE: 0.376717739035863, MSE: 0.14191625490429255, Pearson: 0.8933051324301862, Spearman: 0.8893694248450851, CI: 0.890743195275649
